In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bisect import bisect_right
import functions as f
import winsound
import time
import itertools

def shift_time(timestamp_str, minutes=0, seconds=0, hours=0):
    # Only time (HH:MM:SS)
    if len(timestamp_str) == 8:
        h = int(timestamp_str[0:2])
        m = int(timestamp_str[3:5])
        s = int(timestamp_str[6:8])
        total = h * 3600 + m * 60 + s + hours * 3600 + minutes * 60 + seconds
        total %= 86400  # Wrap around 24h
        return f"{total // 3600:02}:{(total % 3600) // 60:02}:{total % 60:02}"

    # Only date (YYYY-MM-DD)
    elif len(timestamp_str) == 10:
        y, mo, d = map(int, timestamp_str.split("-"))
        t = timedelta(hours=hours, minutes=minutes, seconds=seconds)
        dt = datetime(y, mo, d) + t
        return dt.strftime("%Y-%m-%d")

    # Full datetime (YYYY-MM-DD HH:MM:SS)
    else:
        y = int(timestamp_str[0:4])
        mo = int(timestamp_str[5:7])
        d = int(timestamp_str[8:10])
        h = int(timestamp_str[11:13])
        m = int(timestamp_str[14:16])
        s = int(timestamp_str[17:19])
        total = (datetime(y, mo, d, h, m, s) +
                 timedelta(hours=hours, minutes=minutes, seconds=seconds))
        return total.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
df = pd.read_csv('Data/backtest_model_results.csv')

entry_cols_df = df.filter(regex="^entry_")
entry_cols_dicts = entry_cols_df.to_dict(orient='records')


In [ ]:
def combinations_backtesting_with_dataframe(combi_entry_STD_gen, combi_entry_pctB_gen, sl_gen, tp_gen):

    save_file_name = f"___STD_{combi_entry_STD_gen}____pctB_{combi_entry_pctB_gen}____sl_{sl_gen}____tp_{tp_gen}___"

    print(save_file_name)

    is_last_candle = df['is_last_candle'].values

    entry_datetime = df['entry_datetime'].values
    entry_symbol = df['entry_symbol'].values
    entry_date = df['entry_date'].values
    entry_time = df['entry_time'].values
    combi_entry_STD = df['combi_entry_STD'].values
    combi_entry_pctB = df['combi_entry_pctB'].values

    exit_sl1 = df['exit_sl1'].values
    exit_sl1_pnl = df['exit_sl1_pnl'].values
    exit_sl1_index = df['exit_sl1_index'].values
    exit_sl1_time = df['exit_sl1_time'].values
    exit_sl2 = df['exit_sl2'].values
    exit_sl2_pnl = df['exit_sl2_pnl'].values
    exit_sl2_index = df['exit_sl2_index'].values
    exit_sl2_time = df['exit_sl2_time'].values
    exit_sl3 = df['exit_sl3'].values
    exit_sl3_pnl = df['exit_sl3_pnl'].values
    exit_sl3_index = df['exit_sl3_index'].values
    exit_sl3_time = df['exit_sl3_time'].values
    exit_tp1 = df['exit_tp1'].values
    exit_tp1_pnl = df['exit_tp1_pnl'].values
    exit_tp1_index = df['exit_tp1_index'].values
    exit_tp1_time = df['exit_tp1_time'].values
    exit_tp2 = df['exit_tp2'].values
    exit_tp2_pnl = df['exit_tp2_pnl'].values
    exit_tp2_index = df['exit_tp2_index'].values
    exit_tp2_time = df['exit_tp2_time'].values
    exit_tp3 = df['exit_tp3'].values
    exit_tp3_pnl = df['exit_tp3_pnl'].values
    exit_tp3_index = df['exit_tp3_index'].values
    exit_tp3_time = df['exit_tp3_time'].values
    entry_dicts = list(entry_cols_dicts)

    length_df = -len(df)

    # next trade is after the exit candle
    last_trade_exit_time = shift_time(entry_datetime[length_df], seconds=-5) 
    previous_date = None

    ddff = []

    for i in range(length_df, 0):

        # new day
        if previous_date != entry_date[i]:
            print("Current Date: =====", entry_date[i], "=====")

        if is_last_candle[i]: 

            if entry_datetime[i] > last_trade_exit_time:

                STD_cond = combi_entry_STD[i] - 0.5 <= combi_entry_STD_gen <= combi_entry_STD[i] + 0.5
                pctB = combi_entry_pctB[i] - 0.05 <= combi_entry_pctB_gen <= combi_entry_pctB[i] + 0.05

                if STD_cond and pctB:

                    print("---- Entry ----", entry_symbol[i][-7:])
                    print(entry_datetime[i])

                    entry_dict = entry_dicts[i]

                    # ===== sl_gen ===== 
                    sl_index = {'sl1': exit_sl1_index[i], 'sl2': exit_sl2_index[i], 'sl3': exit_sl3_index[i]}[sl_gen]
                    sl_pnl = {'sl1': exit_sl1_pnl[i], 'sl2': exit_sl2_pnl[i], 'sl3': exit_sl3_pnl[i]}[sl_gen]
                    sl_price = {'sl1': exit_sl1[i], 'sl2': exit_sl2[i], 'sl3': exit_sl3[i]}[sl_gen]
                    sl_time = {'sl1': exit_sl1_time[i], 'sl2': exit_sl2_time[i], 'sl3': exit_sl3_time[i]}[sl_gen]

                    # ===== tp_gen ===== 
                    tp_index = {'tp1': exit_tp1_index[i], 'tp2': exit_tp2_index[i], 'tp3': exit_tp3_index[i]}[tp_gen]
                    tp_pnl = {'tp1': exit_tp1_pnl[i], 'tp2': exit_tp2_pnl[i], 'tp3': exit_tp3_pnl[i]}[tp_gen]
                    tp_price = {'tp1': exit_tp1[i], 'tp2': exit_tp2[i], 'tp3': exit_tp3[i]}[tp_gen]
                    tp_time = {'tp1': exit_tp1_time[i], 'tp2': exit_tp2_time[i], 'tp3': exit_tp3_time[i]}[tp_gen]

                    # Decide which exit happened first – SL or TP
                    if sl_index <= tp_index:
                        print(f"---- sl ----")
                        print(sl_time)
                        last_trade_exit_time = sl_time

                        sltp_data = {'exit_price': sl_price, 'pnl': sl_pnl, 'candle_in_trade': sl_index, 'exit_time': sl_time}
                    else:
                        print(f"---- tp ----")
                        print(tp_time)
                        last_trade_exit_time = tp_time

                        sltp_data = {'exit_price': tp_price, 'pnl': tp_pnl, 'candle_in_trade': tp_index, 'exit_time': tp_time}

                    # Merge entry and sltp_data into one row
                    entry_dict.update(sltp_data)
                    ddff.append(entry_dict)

        previous_date = entry_date[i]

    # Convert to DataFrame
    ddff = pd.DataFrame(ddff)

    # Ensure required columns exist
    if 'pnl' not in ddff.columns:
        ddff['pnl'] = 0.0
    if 'candle_in_trade' not in ddff.columns:
        ddff['candle_in_trade'] = 0

    # ==================================================
    # ==================================================

    total_profit = round(ddff['pnl'].sum(), 2)
    print("Profit       :", total_profit, "%")

    total_trades = len(ddff) + 1
    print("Total Trades :", total_trades)

    avg_gain_per_trade = round(total_profit / total_trades, 2)
    print("Avg Gain per trade   :", avg_gain_per_trade, "%")

    avg_bar_per_trade = ddff['candle_in_trade'].sum() / total_trades
    print("Avg Bar per trade    :", round(avg_bar_per_trade, 0))

    winning_trades = len(ddff[ddff['pnl'] > 0]) + 1
    percent_profitable = ((winning_trades - total_trades)/total_trades) * 100 + 100
    print("Percent Profitable   :", round(percent_profitable, 2), "%")

    lossing_trades = len(ddff[ddff['pnl'] < 0]) + 1
    profit_factor = winning_trades/lossing_trades
    print("Profit Factor        :", round(profit_factor, 2))

    def calculate_max_drawdown(ddff):
        df = ddff.copy()
        df['Cumulative_return'] = df['pnl'].cumsum()
        df['Max Peak'] = df['Cumulative_return'].cummax()
        df['Drawdown'] = df['Cumulative_return'] - df['Max Peak']
        max_drawdown = df['Drawdown'].min()
        return round(max_drawdown, 2)

    max_drawdown = calculate_max_drawdown(ddff)
    print("Max Drawdown :", max_drawdown, "\n")

    output_string = f"""
    Profit                        :  {total_profit} ________________
    Total Trades              :  {total_trades} ________________
    Avg Gain per Trade   :  {avg_gain_per_trade} % _____________
    Avg Bar per Trade    :  {round(avg_bar_per_trade, 0)} _________________
    Percent Profitable    :  {round(percent_profitable, 2)} % ____________
    Profit Factor              :  {round(profit_factor, 2)} _________________
    Max Drawdown       :  {max_drawdown} _______________
    """

    # f.plot_cumulative_return(ddff, output_string, save_file_name, save_fig=False, df_plot_boolian=False)

    return save_file_name, total_profit, total_trades, avg_gain_per_trade, avg_bar_per_trade, percent_profitable, profit_factor, max_drawdown, ddff

combi, tp, tt, agpt, abpt, pp, pf, md, ddff = combinations_backtesting_with_dataframe(12.0, 0.18, "sl3", "tp3")

In [ ]:
ddff

In [ ]:

def generate_combinations(df, config, sl_list=None, tp_list=None):
    all_ranges = []
    column_order = []

    # From DataFrame columns
    for col in config:
        col_config = config[col]
        round_val = col_config.get("round", 2)
        dtype = df[col].dropna().dtype

        if pd.api.types.is_bool_dtype(dtype):
            steps = sorted(df[col].dropna().unique().tolist())
            if not steps:
                steps = [True, False]
            all_ranges.append(steps)
            column_order.append(col)

        elif pd.api.types.is_numeric_dtype(dtype):
            col_min = df[col].min()
            col_max = df[col].max()

            if "step" not in col_config:
                raise ValueError(f"`step` is required for numeric column '{col}'")

            step = col_config["step"]
            steps = [round(i, round_val) for i in np.arange(col_min, col_max + step, step)]
            all_ranges.append(steps)
            column_order.append(col)

        elif pd.api.types.is_string_dtype(dtype) or pd.api.types.is_categorical_dtype(dtype) or pd.api.types.is_object_dtype(dtype):
            steps = df[col].dropna().unique().tolist()
            all_ranges.append(steps)
            column_order.append(col)

        else:
            raise ValueError(f"Unsupported column type for column '{col}'")

    # Add sl and tp as string categories (if provided)
    if sl_list:
        all_ranges.append(sl_list)
        column_order.append("sl")
    if tp_list:
        all_ranges.append(tp_list)
        column_order.append("tp")

    combinations = [
        dict(zip(column_order, combo))
        for combo in itertools.product(*all_ranges)
    ]
    return combinations

columns_to_keep = [col for col in df.columns if "combi_" in col]
print(columns_to_keep)

config = {
    'combi_entry_STD' :             {'step': 1,'round': 2},
    'combi_entry_pctB' :            {'step': 0.1,'round': 2}
}

combinations = generate_combinations(df, config, 
                                     sl_list=['sl1', 'sl2', 'sl3'], 
                                     tp_list=['tp1', 'tp2', 'tp3'])

print(len(combinations))

# --------------------------------------

combinations_df = pd.DataFrame(columns=['combi', 'total_profit', 'total_trades', 'avg_gain_per_trade', 'avg_bar_per_trade', 'percent_profitable', 'profit_factor', 'max_drawdown'])

n = 0
for combi in combinations:
    n += 1
    print(n)

    combi_entry_STD = combi['combi_entry_STD']
    combi_entry_pctB = combi['combi_entry_pctB']
    sl = combi['sl']
    tp = combi['tp']

    combi, tp, tt, agpt, abpt, pp, pf, md, ddff = combinations_backtesting_with_dataframe(combi_entry_STD, combi_entry_pctB, sl, tp)

    inter_combi = pd.DataFrame({'combi': [combi], 'total_profit': [tp], 'total_trades': [tt], 'avg_gain_per_trade': [agpt], 'avg_bar_per_trade': [abpt], 'percent_profitable': [pp], 'profit_factor': [pf], 'max_drawdown': [md]})
    combinations_df = pd.concat([combinations_df, inter_combi], ignore_index=True)

filtered_df = combinations_df.copy()

In [ ]:
combinations_df.head()

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
sorted_df = filtered_df.sort_values(
    by=['avg_gain_per_trade'],        # columns to sort by
    ascending=[False]     # sort col1 ascending, col2 descending
)
sorted_df

In [ ]:
sorted_df = filtered_df.sort_values(
    by=['total_profit', 'avg_gain_per_trade', 'max_drawdown'],        # columns to sort by
    ascending=[False, False, False]     # sort col1 ascending, col2 descending
)
sorted_df